In [19]:
import PyPDF2
import glob
import numpy as np
import pdfplumber
import pandas as pd
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [4]:
def text_extraction(element):
    
    line_text = element.get_text()
    
    line_formats = []
    for text_line in element:
        
        if isinstance(text_line, LTTextContainer):
            
            for character in text_line:
                
                if isinstance(character, LTChar):
                    
                    line_formats.append(character.fontname)
                    
                    line_formats.append(character.size)
                    
    format_per_line = list(set(line_formats))
    
    return (line_text, format_per_line)

def analyst_executives(path):
    
    time=0
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    text=[]
    executives_name=[]
    analysts_name=[]

    executives_title=[]
    analysts_title=[]

    executives_index=0
    analysts_index=0

    for pagenum, page in enumerate(extract_pages(path)):
        
        for _, element in enumerate(page):

            if pagenum == 2:

                if isinstance(element, LTTextContainer):
                    line_text, format_per_line = text_extraction(element)

                text.append(line_text)

                if line_text == "EXECUTIVES\n":
                    executives_index+=1

                if line_text == 'ANALYSTS\n':
                    executives_index=0

                if executives_index >= 1:
                    executives_name.append(line_text.split("\n")[0])
                    executives_title.append(line_text.split("\n")[1])

                if line_text == 'ANALYSTS\n':
                    analysts_index+=1
                    executives_index=0

                if 'S&P Global Market Intelligence' in line_text:
                    analysts_index=0

                if analysts_index >= 1:
                    analysts_name.append(line_text.split("\n")[0])    
                    analysts_title.append(line_text.split("\n")[1])

        if pagenum == 4:
            break

    ppl=defaultdict(lambda: defaultdict(str))

    for i in range(1, len(analysts_name), 1):

        ppl['analysts'][analysts_name[i]]=analysts_title[i]

    for i in range(1, len(executives_name), 1):

        ppl['executives'][executives_name[i]]=executives_title[i]
    
    return ppl

def processing(path):
    
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    time=0

    for pagenum, page in enumerate(extract_pages(path)):

        test=[]

        for _, element in enumerate(page):

            if isinstance(element, LTTextContainer):

                line_text, format_per_line = text_extraction(element)

            test.append(line_text)

            if pagenum == 0:
                if _ == 0:
                    time=line_text
                    time=time[61:100]

            if line_text == 'Presentation\n':
                presentation_index = presentation_index + 1

            if line_text == 'Question and Answer\n':
                Question_and_Answer_index = Question_and_Answer_index + 1

            if Question_and_Answer_index == 2:
                break

            if presentation_index == 2:
                presentation_speech.append(line_text)
    
    return time, presentation_speech

def speech_dict(presentation_speech, executives):

    speech_people=defaultdict(lambda: defaultdict(str))
    speech_index_dict=defaultdict(lambda: defaultdict(str))
    speech_index=0

    for _, text in enumerate(presentation_speech):
        text=text.split("\n")
        for executives in executives_people:
            if executives in text[0]:
                speech_people[str("Presentation_")+str(speech_index)]=executives
                speech_index_dict[str("Presentation_")+str(speech_index)]=_+1
                speech_index+=1

    # ================================================================================
    speech_speech_index_list=[]

    for i in speech_index_dict.items():

        speech_speech_index_list.append(i[1])

    speech_speech_index_list.append(len(presentation_speech))

    # ================================================================================
    length=len(speech_index_dict.items())
    speech_speech_index_all=defaultdict(lambda: defaultdict(list))

    for i in range(length):

        rang=range(speech_speech_index_list[i], speech_speech_index_list[i+1], 1)

        list_=[x for x in rang]

        speech_speech_index_all[str("Presentation_")+str(i)]=list_

    speech_speech_dict=defaultdict(lambda: defaultdict(list))
    speech_index=0

    for i in speech_speech_index_all.items():

        for j in i[1]:

            speech_speech_dict[i[0]][j].append(presentation_speech[j])
            
    return speech_speech_dict, speech_people

In [5]:
files=glob.glob("/Volumes/4TB/ConferenceCall/Tesla/Earningcall/*.pdf")
path=files[0]
print(path)
pdf=open(path, 'rb')
pdfReader=PyPDF2.PdfReader(pdf)

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2011 Earnings Call, May 04, 2011.pdf


In [6]:
ppl=analyst_executives(path)

ppl['executives'].items()
executives_people=[elem[0] for elem in ppl['executives'].items()]
executives_title=[elem[1] for elem in ppl['executives'].items()]

ppl['analysts'].items()
analysts_people=[elem[0] for elem in ppl['analysts'].items()]
analysts_title=[elem[1] for elem in ppl['analysts'].items()]

In [7]:
time, presentation_speech=processing(path)

In [8]:
speech_speech_dict, speech_people = speech_dict(presentation_speech, executives_people)

In [14]:
def data_processing_pre(path):
    
    pdf=open(path, 'rb')
    pdfReader=PyPDF2.PdfReader(pdf)

    time, presentation_speech=processing(path)

    ppl=analyst_executives(path)

    ppl['executives'].items()
    executives_people=[elem[0] for elem in ppl['executives'].items()]
    executives_title=[elem[1] for elem in ppl['executives'].items()]

    ppl['analysts'].items()
    analysts_people=[elem[0] for elem in ppl['analysts'].items()]
    analysts_title=[elem[1] for elem in ppl['analysts'].items()]
    
    speech_speech_dict, speech_people =\
    speech_dict(presentation_speech, executives_people)
    
    return time, path.split("/")[-1], ppl, speech_people, speech_speech_dict

In [15]:
data_processing_pre(path)

('Wednesday, May 04, 2011 9:00 PM GMT\nS&P',
 'Tesla Motors, Inc., Q1 2011 Earnings Call, May 04, 2011.pdf',
 defaultdict(<function __main__.analyst_executives.<locals>.<lambda>()>,
             {'analysts': defaultdict(str,
                          {'Adam Michael Jonas': 'Morgan Stanley, Research Division',
                           'Andrea Susan James': 'Dougherty & Company LLC,',
                           'Daniel V. Galves': 'Deutsche Bank AG, Research',
                           'Himanshu A. Patel': 'JP Morgan Chase & Co, Research',
                           'Patrick Kenehan Archambault': 'Goldman Sachs Group Inc.,',
                           'Unknown Analyst': ''}),
              'executives': defaultdict(str,
                          {'Deepak Ahuja': 'Former Chief Financial Officer',
                           'Elon R. Musk': 'Co-Founder, CEO, Director &',
                           'Jeffrey K. Evanson': 'Former Vice President of Investor'})}),
 defaultdict(<function __mai

In [28]:
data=defaultdict(lambda: defaultdict(dict))

for path in files[:]:
    
    print(path)
    
    time, document, ppl, speech_people, speech_speech_dict=data_processing_pre(path)
    
    data[time][document]["people"]=ppl
    data[time][document]["presentation_people"]=speech_people
    data[time][document]["presentation_people_speech"]=speech_speech_dict

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2011 Earnings Call, May 04, 2011.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2012 Earnings Call, May 09, 2012.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2013 Earnings Call, May 08, 2013.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2014 Earnings Call, May 07, 2014.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2015 Earnings Call, May 06, 2015.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2016 Earnings Call, May 04, 2016.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q2 2011 Earnings Call, Aug 03, 2011.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q2 2012 Earnings Call, Jul 25, 2012.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q2 2013 Earnings Call, Aug 07, 2013.pdf
/Volumes/4TB/ConferenceCall/Tesla/Earningcall/

In [37]:
data_table=\
pd.DataFrame([
              col1, col2, col3, d3
             ]
              for col1, d1 in data.items()\
              for col2, d2 in d1.items()\
              for col3, d3 in d2.items()
            )

In [38]:
data_table=data_table.rename(columns={0: 'time', 1: 'company', 2: 'chara', 3: 'text'})

In [39]:
data_table

,time,company,chara,text
0,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",people,{'analysts': {'Adam Michael Jonas': 'Morgan St...
1,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",presentation_people,"{'Presentation_0': 'Jeffrey K. Evanson', 'Pres..."
2,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",presentation_people_speech,"{'Presentation_0': {4: ['Thank you, Tyrone, an..."
3,"Wednesday, May 09, 2012 9:30 PM GMT\nS&P","Tesla Motors, Inc., Q1 2012 Earnings Call, May...",people,{'analysts': {'Adam Michael Jonas': 'Morgan St...
4,"Wednesday, May 09, 2012 9:30 PM GMT\nS&P","Tesla Motors, Inc., Q1 2012 Earnings Call, May...",presentation_people,"{'Presentation_0': 'Jeffrey K. Evanson', 'Pres..."
...,...,...,...,...
151,"Wednesday, January 25, 2023 10:30 PM GM","Tesla, Inc., Q4 2022 Earnings Call, Jan 25, 20...",presentation_people,{'Presentation_0': 'Elon R. Musk'}
152,"Wednesday, January 25, 2023 10:30 PM GM","Tesla, Inc., Q4 2022 Earnings Call, Jan 25, 20...",presentation_people_speech,"{'Presentation_0': {6: ['Thank you, Martin. So..."
153,"Wednesday, February 22, 2017 10:30 PM G","Tesla Motors, Inc., Q4 2016 Earnings Call, Oct...",people,{'analysts': {'David J. Tamberrino': 'Goldman ...
154,"Wednesday, February 22, 2017 10:30 PM G","Tesla Motors, Inc., Q4 2016 Earnings Call, Oct...",presentation_people,"{'Presentation_0': 'Jeffrey K. Evanson', 'Pres..."


In [40]:
table=\
pd.pivot(data_table,
         values=['text'],
         index=['time', 'company'],
         columns=['chara'])

In [41]:
table_=table['text'].reset_index(drop=False)

In [42]:
table_=table['text'].reset_index(drop=False)
table_["date"]=table_["time"].apply(lambda x: x.split(',')[1])
table_["year"]=\
table_["time"].apply(lambda x: x.split(',')[2][0:5])
table_["datetime"]=\
table_["time"].apply(lambda x: x.split(',')[2][5:11])
table_["quarter"]=\
table_["company"].apply(lambda x: x.split(',')[2][:3])
table_["year_quarter"]=\
table_["company"].apply(lambda x: x.split(',')[2][3:9])
table_=table_.sort_values(['year_quarter', 'quarter'])

In [43]:
table_.columns

Index(['time', 'company', 'people', 'presentation_people',
       'presentation_people_speech', 'date', 'year', 'datetime', 'quarter',
       'year_quarter'],
      dtype='object', name='chara')

In [44]:
table_=\
table_[['date', 'datetime', 'year', 'year_quarter', 'quarter',
        'time', 'company', 'people', 'presentation_people',
        'presentation_people_speech']].reset_index(drop=True)

In [46]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_presentation.csv'

In [47]:
table_.to_csv(path_output, index = False)